In [22]:
import cv2 
import numpy as np

The compute_disparity_map function performs stereo block matching to generate a disparity map from two rectified grayscale images. 
It compares local patches around each pixel in the left image with patches in the right image using either the Sum of Absolute Differences (SAD) or Sum of Squared Differences (SSD) as a matching metric. The function iterates over each pixel, searches across a specified disparity range, and records the disparity corresponding to the best match. This approach is fundamental in stereo vision to infer depth from image pairs.


In [23]:

def compute_disparity_map(Il, Ir, window_size, method, max_disparity=192):

    h, w = Il.shape
    half_w = window_size // 2
    disparity_map = np.zeros((h, w), dtype=np.uint8) 
    
    padded_Il = cv2.copyMakeBorder(Il, half_w, half_w, half_w, half_w, cv2.BORDER_CONSTANT, 0) # pad the left image
    padded_Ir = cv2.copyMakeBorder(Ir, half_w, half_w, half_w, half_w, cv2.BORDER_CONSTANT, 0) # pad the right image

    for y in range(half_w, h + half_w):
        for x in range(half_w, w + half_w):
            best_offset = 0
            best_score = float('inf')

            left_patch = padded_Il[y - half_w:y + half_w + 1, x - half_w:x + half_w + 1]

            for offset in range(max_disparity):
                xr = x - offset # right image x coordinate
                if xr - half_w < 0:
                    continue  # out of bounds on right image

                right_patch = padded_Ir[y - half_w:y + half_w + 1, xr - half_w:xr + half_w + 1] # get the right patch
                
                if method == "SAD":
                    score = np.sum(np.abs(left_patch - right_patch))
                elif method == "SSD":
                    score = np.sum((left_patch - right_patch) ** 2)
                else:
                    raise ValueError("Unknown method: choose 'SAD' or 'SSD'")

                if score < best_score: # update best score
                    best_score = score
                    best_offset = offset

            disparity_map[y - half_w, x - half_w] = best_offset # store the best offset in the disparity map

    return disparity_map


In [24]:
# Load stereo images
imgl = cv2.imread('stereo_materials/l1.png', cv2.IMREAD_GRAYSCALE)
imgr = cv2.imread('stereo_materials/r1.png', cv2.IMREAD_GRAYSCALE)

# Set parameters
window_sizes = [1, 5, 9]
methods = ["SAD", "SSD"]

for window_size in window_sizes:
    for method in methods:
        print(f"Processing {method} with window size {window_size}...")
        
        disparity_map = compute_disparity_map(imgl, imgr, window_size, method)
        
        # Normalize for visualization
        disp_display = cv2.normalize(disparity_map, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX)
        disp_display = np.uint8(disp_display)
        
        # Save image
        output_filename = f"stereo_materials/disparity_{method}_w{window_size}.png"
        cv2.imwrite(output_filename, disp_display)

        #Show the result
        cv2.imshow(f'Disparity {method} Window {window_size}', disp_display)
        cv2.waitKey(500)  # Show each for 0.5 seconds

cv2.destroyAllWindows()

print("All disparity maps saved in 'stereo_materials/' folder.")


Processing SAD with window size 1...
Processing SSD with window size 1...
Processing SAD with window size 5...
Processing SSD with window size 5...
Processing SAD with window size 9...
Processing SSD with window size 9...
All disparity maps saved in 'stereo_materials/' folder.
